In [2]:
# unix time: https://www.unixtimestamp.com/
import datetime  # Convert to unix time
import math
import os
import pickle
import time  # Convert to unix time
import warnings

import dask.dataframe as dd  # similar to pandas
import matplotlib.pylab as plt
# if numpy is not installed already : pip3 install numpy
import numpy as np  # Do aritmetic operations on arrays
import pandas as pd  # pandas to create small dataframes
import seaborn as sns  # Plots
# to install xgboost: pip3 install xgboost
# if it didnt happen check install_xgboost.JPG
import xgboost as xgb
from matplotlib import rcParams  # Size of plots
from sklearn.cluster import KMeans, MiniBatchKMeans  # Clustering
# to install sklearn: pip install -U scikit-learn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (mean_absolute_error,
                             mean_absolute_percentage_error,
                             mean_squared_error, r2_score)

warnings.filterwarnings("ignore")

# Data Load

In [3]:
base_year = 2018
base_month_count = 12

In [4]:
def load_data():
    months_frame = []
    months_groupby = []
    for i in range(1,base_month_count+1):
        tmp_frame = pd.read_parquet(f'preprocessing_yellow_tripdata_{base_year+1}_{i}.parquet',engine='pyarrow')
        tmp_groupby = tmp_frame[['PULocationID','pickup_bins','trip_distance']].groupby(['PULocationID','pickup_bins']).count()
       
        months_frame.append(tmp_frame)
        months_groupby.append(tmp_groupby)
    return months_frame, months_groupby

In [5]:
months_frame, months_groupby = load_data()

In [6]:
months_frame[0][months_frame[0]['pickup_bins']<0]

,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount,tip_amount,trip_times,pickup_times,Speed,pickup_bins
39,1.0,1.36,4,114,9.30,0.00,9.016667,1.546237e+09,9.049908,-52
16350,1.0,4.92,13,48,26.30,3.00,29.550000,1.546250e+09,9.989848,-31
17826,1.0,0.49,13,231,6.80,2.00,2.550000,1.546246e+09,11.529412,-37
101380,2.0,0.62,41,74,6.30,0.00,4.583333,1.546253e+09,8.116364,-25
101508,2.0,5.26,41,48,19.30,0.00,15.016667,1.546255e+09,21.016648,-23
...,...,...,...,...,...,...,...,...,...,...
6714646,1.0,2.55,262,143,11.30,0.00,11.233333,1.230737e+09,13.620178,-525886
6795253,1.0,3.85,263,168,14.80,0.00,12.866667,1.546230e+09,17.953368,-63
6796436,2.0,0.38,263,141,5.16,0.86,1.066667,1.546241e+09,21.375000,-45
6827621,1.0,1.05,263,263,7.80,0.00,8.016667,1.230741e+09,7.858628,-525878


## 뉴욕 지역

In [7]:
taxi_zone_df = pd.read_csv('taxi_zone_lookup.csv')

In [8]:
region = "Manhattan"
nyc_region = taxi_zone_df[taxi_zone_df['Borough'] == region]
nyc_region_number = nyc_region['LocationID']
nyc_regions_cnt = len(nyc_region)

In [9]:
months_frame

[         passenger_count  trip_distance  PULocationID  DOLocationID  \
 0                    1.0           3.80             4           246   
 1                    1.0           2.70             4            87   
 2                    1.0           2.30             4           233   
 3                    1.0           4.70             4           263   
 4                    1.0           3.90             4           261   
 ...                  ...            ...           ...           ...   
 6950960              NaN           6.26           263           232   
 6950961              NaN          15.34           263            98   
 6950962              NaN           9.16           263            32   
 6950963              NaN          18.53           263           191   
 6950964              NaN           4.43           263           159   
 
          total_amount  tip_amount  trip_times  pickup_times      Speed  \
 0               26.30        0.00   40.583333  1.546269e+0

# Smoothing

In [10]:
# number of 10min indices for jan 2019= 24*31*60/10 = max_pickup_bins_len
interval = 10
days = [31,28,31,30,31,30,31,31,30,31,30,31]
pickup_bins_len = []

for day in days:
    pickup_bins_len.append(int(24*60*day/interval))
max_pickup_bins_len = max(pickup_bins_len)
print(max_pickup_bins_len)

4464


In [11]:
# Fills a value of zero for every bin where no pickup data is present 
# the count_values: number pickps that are happened in each region for each 10min intravel
# there wont be any value if there are no picksups.
# values: number of unique bins

# for every 10min intravel(pickup_bin) we will check it is there in our unique bin,
# if it is there we will add the count_values[index] to smoothed data
# if not we add smoothed data (which is calculated based on the methods that are discussed in the above markdown cell)
# we finally return smoothed data
def smoothing(count_values,values):
    smoothed_regions=[] # stores list of final smoothed values of each reigion
    ind=0
    repeat=0 
    smoothed_value=0
    for r in range(1,nyc_regions_cnt+1):
        smoothed_bins=[] #stores the final smoothed values
        repeat=0
        for i in range(max_pickup_bins_len):
            if repeat!=0: # prevents iteration for a value which is already visited/resolved
                repeat-=1
                continue
            if i in values[r-1]: #checks if the pickup-bin exists 
                smoothed_bins.append(count_values[ind-1]) # appends the value of the pickup bin if it exists
            else:
                if i!=0:
                    right_hand_limit=0
                    for j in range(i,max_pickup_bins_len):
                        if  j not in values[r-1]: #searches for the left-limit or the pickup-bin value which has a pickup value
                            continue
                        else:
                            right_hand_limit=j
                            break
                    if right_hand_limit==0:
                    #Case 1: When we have the last/last few values are found to be missing,hence we have no right-limit here
                        smoothed_value=count_values[ind-1]*1.0/((max_pickup_bins_len-1-i)+2)*1.0                               
                        for j in range(i,max_pickup_bins_len):                              
                            smoothed_bins.append(math.ceil(smoothed_value))
                        smoothed_bins[i-1] = math.ceil(smoothed_value)
                        repeat=(max_pickup_bins_len-1-i)
                        ind-=1
                    else:
                    #Case 2: When we have the missing values between two known values
                        smoothed_value=(count_values[ind-1]+count_values[ind])*1.0/((right_hand_limit-i)+2)*1.0             
                        for j in range(i,right_hand_limit+1):
                            smoothed_bins.append(math.ceil(smoothed_value))
                        smoothed_bins[i-1] = math.ceil(smoothed_value)
                        repeat=(right_hand_limit-i)
                else:
                    #Case 3: When we have the first/first few values are found to be missing,hence we have no left-limit here
                    right_hand_limit=0
                    for j in range(i,max_pickup_bins_len):
                        if  j not in values[r-1]:
                            continue
                        else:
                            right_hand_limit=j
                            break
                    smoothed_value=count_values[ind]*1.0/((right_hand_limit-i)+1)*1.0
                    for j in range(i,right_hand_limit+1):
                            smoothed_bins.append(math.ceil(smoothed_value))
                    repeat=(right_hand_limit-i)
            ind+=1
        smoothed_regions.extend(smoothed_bins)
    return smoothed_regions

In [12]:
def return_unq_pickup_bins(frame):
    values = []
    for i in nyc_region_number.values:
    # for i in range(1,266):
        new = frame[frame['PULocationID'] == i]
        list_unq = list(set(new['pickup_bins']))
        list_unq.sort()
        values.append(list_unq)
    return values

In [13]:
months_unique = []
for frame in months_frame:
    months_unique.append(return_unq_pickup_bins(frame))

In [14]:
months_smooth = []
for groupby, unique in zip(months_groupby, months_unique):
    # smoothing을 할 것인가 filling을 할것인가
    months_smooth.append(smoothing(groupby['trip_distance'].values,unique))

# Making list of all the values of pickup data in every bin for a period of 3 months and storing them region-wise 
regions_cum = []


# number of 10min indices for jan 2019= 24*31*60/10 = 4464      # pickup_bins_len[0]
# number of 10min indices for jan 2020 = 24*31*60/10 = 4464     # pickup_bins_len[0]
# number of 10min indices for feb 2020 = 24*29*60/10 = 4176     # pickup_bins_len[1]
# number of 10min indices for march 2020 = 24*31*60/10 = 4464   # pickup_bins_len[2]
# regions_cum: it will contain 40 lists, each list will contain 4464+4176+4464 values which represents the number of pickups 
# that are happened for three months in 2016 data

# nyc_regions_cnt개의 맨허튼 지역
for i in range(1,nyc_regions_cnt+1):
    cum = []
    for index, smooth in enumerate(months_smooth):
        cum += smooth[pickup_bins_len[index]*(i-1):pickup_bins_len[index]*i]
    regions_cum.append(cum)

print(len(regions_cum))
print(len(regions_cum[0]))


69
52560


# Modeling

## 회귀 모델

In [15]:

# Preparing data to be split into train and test, The below prepares data in cumulative form which will be later split into test and train
# number of 10min indices for jan 2019= 24*31*60/10 = 4464      # pickup_bins_len[0]
# number of 10min indices for jan 2020 = 24*31*60/10 = 4464     # pickup_bins_len[0]
# number of 10min indices for feb 2020 = 24*29*60/10 = 4176     # pickup_bins_len[1]
# number of 10min indices for march 2020 = 24*31*60/10 = 4464   # pickup_bins_len[2]
# regions_cum: it will contain 40 lists, each list will contain 4464+4176+4464 values which represents the number of pickups 
# that are happened for three months in 2016 data

# print(len(regions_cum))
# 265
# print(len(regions_cum[0]))
# 4368


# we take number of pickups that are happened in last 5 intravels
number_of_time_stamps = 5

# output varaible
# it is list of lists
# it will contain number of pickups 4368 for each cluster
# len(regions_cum[0]) == 4368
output = []

sum(pickup_bins_len[:base_month_count])
# 우리 데이터
# len(regions_cum[0]) - 5(:= # of colunms)
# 4368 - 5 = 4363
# 13104 - 5 = 13099

# tsne_lat will contain 13104-5=13099 times lattitude of cluster center for every cluster
# Ex: [[cent_lat 13099times],[cent_lat 13099times], [cent_lat 13099times].... 40 lists]
# it is list of lists
# tsne_lat = []

# tsne_lon will contain 13104-5=13099 times logitude of cluster center for every cluster
# Ex: [[cent_long 13099times],[cent_long 13099times], [cent_long 13099times].... 40 lists]
# it is list of lists
# tsne_lon = []

# 우리는 lat, lon 대신에 목적지 ID (PULocationID: 출발지, DOLocationID: 도착지)를 사용할 것이다.
tsne_PULocationID = []

# we will code each day 
# sunday = 0, monday=1, tue = 2, wed=3, thur=4, fri=5, sat=6
# for every cluster we will be adding 13099 values, each value represent to which day of the week that pickup bin belongs to
# it is list of lists
tsne_weekday = []

# its an numbpy array, of shape (523960, 5)
# each row corresponds to an entry in out data
# for the first row we will have [f0,f1,f2,f3,f4] fi=number of pickups happened in i+1th 10min intravel(bin)
# the second row will have [f1,f2,f3,f4,f5]
# the third row will have [f2,f3,f4,f5,f6]
# and so on...
tsne_feature = []


tsne_feature = [0]*number_of_time_stamps
for i in range(1,nyc_regions_cnt+1):
    # tsne_lat.append([kmeans.cluster_centers_[i][0]]*13099) # kmeans.cluster_centers_[i][0] := Coordinates of cluster centers. 클러스트 센터의 상관계수
    # tsne_lon.append([kmeans.cluster_centers_[i][1]]*13099)

    # tsne_PULocationID
    tsne_PULocationID.append([i]*(len(regions_cum[0]) - 5))


    day_of_the_week_dict = {2015: 4, 2016: 5, 2017: 0, 2018:1, 2019:2, 2020:3, 2021:5, 2022:6}
    # jan 1st 2016 is thursday, so we start our day from 4: "(int(k/144))%7+4"
    # our prediction start from 5th 10min intravel since we need to have number of pickups that are happened in last 5 pickup bins
    
    # jan 1st 2020 is tue -> 3
    tsne_weekday.append([int(((int(k/144))%7+day_of_the_week_dict[base_year+1])%7) for k in range(5,sum(pickup_bins_len[:base_month_count]))])

    # jan 1st 2021 is fri -> 5
    # tsne_weekday.append([int(((int(k/144))%7+5)%7) for k in range(5,sum(pickup_bins_len[:3]))])
    # regions_cum is a list of lists [[x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], [x1,x2,x3..x13104], .. 40 lsits]
    
    # 우리 데이터 
    # regions_cum [[x_1,x_2,...,x_{len(regions_cum[0]) - 5}],...265 lists] len(regions_cum[0]) - 5 = 4381
    tsne_feature = np.vstack((tsne_feature, [regions_cum[i-1][r:r+number_of_time_stamps] for r in range(0,len(regions_cum[i-1])-number_of_time_stamps)]))

    output.append(regions_cum[i-1][5:])

tsne_feature = tsne_feature[1:]

In [16]:
print(tsne_feature.shape[0])
print(len(tsne_weekday)*len(tsne_weekday[0]))
print(len(output)*len(output[0]))
print(nyc_regions_cnt*(len(regions_cum[0])-5))
print(len(tsne_PULocationID)*len(tsne_PULocationID[0]))

3626295
3626295
3626295
3626295
3626295


In [17]:
# Getting the predictions of exponential moving averages to be used as a feature in cumulative form

# upto now we computed 8 features for every data point that starts from 50th min of the day
# 1. cluster center lattitude
# 2. cluster center longitude
# 3. day of the week 
# 4. f_t_1: number of pickups that are happened previous t-1th 10min intravel
# 5. f_t_2: number of pickups that are happened previous t-2th 10min intravel
# 6. f_t_3: number of pickups that are happened previous t-3th 10min intravel
# 7. f_t_4: number of pickups that are happened previous t-4th 10min intravel
# 8. f_t_5: number of pickups that are happened previous t-5th 10min intravel

# from the baseline models we said the exponential weighted moving avarage gives us the best error
# we will try to add the same exponential weighted moving avarage at t as a feature to our data
# exponential weighted moving avarage => p'(t) = alpha*p'(t-1) + (1-alpha)*P(t-1) 
alpha=0.3

# it is a temporary array that store exponential weighted moving avarage for each 10min intravel, 
# for each cluster it will get reset
# for every cluster it contains 13104 values
predicted_values=[]

# it is similar like tsne_lat
# it is list of lists
# predict_list is a list of lists [[x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], [x5,x6,x7..x13104], .. 40 lsits]
predict_list = []
tsne_flat_exp_avg = []
for r in range(1,nyc_regions_cnt+1):
    for i in range(0,len(regions_cum[0])):
        if i==0:
            predicted_value= regions_cum[r-1][0]
            predicted_values.append(0)
            continue
        predicted_values.append(predicted_value)
        predicted_value =int((alpha*predicted_value) + (1-alpha)*(regions_cum[r-1][i]))
    predict_list.append(predicted_values[5:])
    predicted_values=[]

In [18]:
len(predict_list)

69

In [19]:
len(predict_list[1])

52555

In [20]:
predict_list[7][27348]

4

In [21]:
# train, test split : 70% 30% split
# Before we start predictions using the tree based regression models we take 3 months of 2016 pickup data 
# and split it such that for every region we have 70% data in train and 30% in test,
# ordered date-wise for every region

sizeof_train_data = int((len(regions_cum[0])-5)*0.7)
sizeof_test_data = int((len(regions_cum[0])-5)*0.3)


print("size of train data :", sizeof_train_data)
print("size of test data :", sizeof_test_data)

size of train data : 36788
size of test data : 15766


In [22]:
# extracting first 91nyc_regions_cnt timestamp values i.e 70% of 13099 (total timestamps) for our training data
train_features =  [tsne_feature[i*(len(regions_cum[0])-5):((len(regions_cum[0])-5)*i+sizeof_train_data)] for i in range(0,nyc_regions_cnt)]

test_features = [tsne_feature[((len(regions_cum[0])-5)*(i))+sizeof_train_data:(len(regions_cum[0])-5)*(i+1)] for i in range(0,nyc_regions_cnt)]

In [23]:
print("Number of data clusters",len(train_features), "Number of data points in trian data", len(train_features[0]), "Each data point contains", len(train_features[0][0]),"features")
print("Number of data clusters",len(train_features), "Number of data points in test data", len(test_features[0]), "Each data point contains", len(test_features[0][0]),"features")

Number of data clusters 69 Number of data points in trian data 36788 Each data point contains 5 features
Number of data clusters 69 Number of data points in test data 15767 Each data point contains 5 features


In [24]:
# extracting first sizeof_train_data timestamp values i.e 70% of 13099 (total timestamps) for our training data

tsne_train_flat_PULocationID = [i[:sizeof_train_data] for i in tsne_PULocationID]
tsne_train_flat_weekday = [i[:sizeof_train_data] for i in tsne_weekday]
tsne_train_flat_output = [i[:sizeof_train_data] for i in output]
tsne_train_flat_exp_avg = [i[:sizeof_train_data] for i in predict_list]

In [25]:
# extracting the rest of the timestamp values i.e 30% of sizeof_train_data + sizeof_test_data (total timestamps) for our test data

tsne_test_flat_PULocationID = [i[sizeof_train_data:] for i in tsne_PULocationID]
tsne_test_flat_weekday = [i[sizeof_train_data:] for i in tsne_weekday]
tsne_test_flat_output = [i[sizeof_train_data:] for i in output]
tsne_test_flat_exp_avg = [i[sizeof_train_data:] for i in predict_list]

In [26]:
# the above contains values in the form of list of lists (i.e. list of values of each region), here we make all of them in one list
train_new_features = []
for i in range(0,nyc_regions_cnt):
    train_new_features.extend(train_features[i])
test_new_features = []
for i in range(0,nyc_regions_cnt):
    test_new_features.extend(test_features[i])

In [27]:
tsne_train_PULocationID = sum(tsne_train_flat_PULocationID, [])
tsne_train_weekday = sum(tsne_train_flat_weekday, [])
tsne_train_output = sum(tsne_train_flat_output, [])
tsne_train_exp_avg = sum(tsne_train_flat_exp_avg,[])

In [28]:
tsne_test_PULocationID = sum(tsne_test_flat_PULocationID, [])
tsne_test_weekday = sum(tsne_test_flat_weekday, [])
tsne_test_output = sum(tsne_test_flat_output, [])
tsne_test_exp_avg = sum(tsne_test_flat_exp_avg,[])

In [29]:
# Preparing the data frame for our train data
columns = ['ft_5','ft_4','ft_3','ft_2','ft_1']
df_train = pd.DataFrame(data=train_new_features, columns=columns) 
# df_train['lat'] = tsne_train_lat
# df_train['lon'] = tsne_train_lon

df_train['PULocationID'] = tsne_train_PULocationID
df_train['weekday'] = tsne_train_weekday
df_train['exp_avg'] = tsne_train_exp_avg

print(df_train.shape)

(2538372, 8)


In [30]:
# Preparing the data frame for our train data
df_test = pd.DataFrame(data=test_new_features, columns=columns) 
# df_test['lat'] = tsne_test_lat
# df_test['lon'] = tsne_test_lon

df_test['PULocationID'] = tsne_test_PULocationID
df_test['weekday'] = tsne_test_weekday
df_test['exp_avg'] = tsne_test_exp_avg
print(df_test.shape)

(1087923, 8)


In [31]:
months_frame[6]

,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount,tip_amount,trip_times,pickup_times,Speed,pickup_bins
0,1.0,1.00,4,114,10.30,0.00,7.950000,1.561909e+09,7.547170,4
1,2.0,5.97,4,239,29.16,4.86,20.600000,1.561909e+09,17.388350,2
2,1.0,4.00,4,230,20.80,0.00,22.083333,1.561908e+09,10.867925,1
3,1.0,2.80,4,88,13.80,0.00,6.666667,1.561909e+09,25.200000,3
4,1.0,3.50,4,255,22.45,5.15,15.100000,1.561910e+09,13.907285,4
...,...,...,...,...,...,...,...,...,...,...
5678283,NaN,2.67,263,239,22.00,0.00,15.766667,1.564543e+09,10.160677,4393
5678284,NaN,2.51,263,151,33.51,0.00,15.000000,1.564544e+09,10.040000,4395
5678285,NaN,3.52,263,168,27.37,0.00,18.766667,1.564550e+09,11.253996,4406
5678286,NaN,3.52,263,74,26.41,0.00,22.316667,1.564550e+09,9.463779,4405


In [32]:
pickup_bins_len

[4464, 4032, 4464, 4320, 4464, 4320, 4464, 4464, 4320, 4464, 4320, 4464]

In [33]:
len(months_frame)

12

In [103]:
input_string = '2019-07-09 22:08:00'
input_time = datetime.datetime.strptime(input_string, '%Y-%m-%d %H:%M:%S')

print('Date-time:', input_time)


def convert_to_unix(s):
    return time.mktime(s.timetuple())

start_pickup_unix = 1546268400 #local time 201901010000

def calculate_bin(time):
    return (int((time-start_pickup_unix)/600)) # !!!!!!!!!!!! 말해보자 -> 33(330분)을 더해주는 이유? [안 더하면 요일계산 잘됨]

unix_time = convert_to_unix(input_time)
print(unix_time)

bin = calculate_bin(unix_time)
print(bin)

def calculate_month(bin):
    month = 0
    check = 0
    while bin>0:
        bin -= pickup_bins_len[check]
        check+=1
        month+=1
    return month

## 입력받은 날짜의 월 계산
target_month = calculate_month(bin)
print("target_month = ", target_month)

## 입력받은 날짜의 월에 해당하는 bin 구하기
target_bin = bin
for i in range(target_month-1):
    target_bin -= pickup_bins_len[i]

print("target_bin = ", target_bin)

weekday = ((int(bin/144))%7+day_of_the_week_dict[base_year+1])%7
print(weekday)

exp_avg_test = predict_list[3][bin] # predict_list[n][bin] -> n-1번 지역의 bin에서의 exp_avg
print(exp_avg_test)

before_frame = months_frame[target_month - 1][['PULocationID','pickup_bins']] #target_frame = 예측년도를 전처리 한 프레임 (jan_base_year_frame)
after_frame = before_frame.loc[(before_frame['pickup_bins'] <target_bin) & (before_frame['pickup_bins']>=target_bin-5)]

# for i in range(1, 2): ## 1~5번째 전 bin의 탑승 수를 맨해튼 모든 지역별로 돌려서 저장 
# ft_1이 10분 전
# 말해볼 것: 달별로 하나씩 뽑아서 평균값으로?

input_list = []

columns = ['ft_5','ft_4','ft_3','ft_2','ft_1']
temp = 0 
for i in nyc_region_number.values:
    temp_list = [0 for i in range(5)]
    temp1_frame = after_frame.loc[(after_frame['PULocationID']==i)]
    globals()['region_{}_frame'.format(i)] = pd.DataFrame(columns = columns)
    for j in range(1,6):
        temp2_frame = temp1_frame.loc[(temp1_frame['pickup_bins'] == target_bin-j)] # ft_1, ft_2, ft_3, ft_4, ft_5 구하기
        temp_list[5-j] = len(temp2_frame)
        # globals()['region_{}_bins_{}'.format(i, j)] = len(temp2_frame)
    eval('region_' + str(i) + '_frame').loc[str(i)+ '번지역'] = temp_list
    eval('region_' + str(i) + '_frame')['weekday'] = weekday
    eval('region_' + str(i) + '_frame')['exp_avg'] = predict_list[temp][bin]
    eval('region_' + str(i) + '_frame')['PULocationID'] = i
    input_list.append(eval('region_' + str(i) + '_frame'))
    temp += 1

Date-time: 2019-07-09 22:08:00
1562677680.0
27348
target_month =  7
target_bin =  1284
2
5


In [73]:
input_list[0]

,ft_5,ft_4,ft_3,ft_2,ft_1,weekday,exp_avg,PULocationID
4번지역,2,1,3,4,4,2,2,4


In [64]:
input_list[3].index[0]

24

In [35]:
region_100_frame

,ft_5,ft_4,ft_3,ft_2,ft_1,weekday,exp_avg
0,43,43,42,45,33,2,44


## Using Linear Regression

In [36]:
# find more about LinearRegression function here http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
# -------------------------
# default paramters
# sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)

# some of methods of LinearRegression()
# fit(X, y[, sample_weight])	Fit linear model.
# get_params([deep])	Get parameters for this estimator.
# predict(X)	Predict using the linear model
# score(X, y[, sample_weight])	Returns the coefficient of determination R^2 of the prediction.
# set_params(**params)	Set the parameters of this estimator.
# -----------------------
# video link: https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/geometric-intuition-1-2-copy-8/
# -----------------------

from sklearn.linear_model import LinearRegression
lr_reg=LinearRegression().fit(df_train, tsne_train_output)

y_pred = lr_reg.predict(df_test)
lr_test_predictions = [round(value) for value in y_pred]
y_pred = lr_reg.predict(df_train)
lr_train_predictions = [round(value) for value in y_pred]

## Using Random Forest Regressor

In [37]:
# Training a hyper-parameter tuned random forest regressor on our train data
# find more about LinearRegression function here http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
# -------------------------
# default paramters
# sklearn.ensemble.RandomForestRegressor(n_estimators=10, criterion=’mse’, max_depth=None, min_samples_split=2, 
# min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, 
# min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)

# some of methods of RandomForestRegressor()
# apply(X)	Apply trees in the forest to X, return leaf indices.
# decision_path(X)	Return the decision path in the forest
# fit(X, y[, sample_weight])	Build a forest of trees from the training set (X, y).
# get_params([deep])	Get parameters for this estimator.
# predict(X)	Predict regression target for X.
# score(X, y[, sample_weight])	Returns the coefficient of determination R^2 of the prediction.
# -----------------------
# video link1: https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/regression-using-decision-trees-2/
# video link2: https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/what-are-ensembles/
# -----------------------

regr1 = RandomForestRegressor(max_features='sqrt',min_samples_leaf=4,min_samples_split=3,n_estimators=40, n_jobs=-1)
regr1.fit(df_train, tsne_train_output)

RandomForestRegressor(max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=3, n_estimators=40, n_jobs=-1)

In [38]:
# Predicting on test data using our trained random forest model 

# the models regr1 is already hyper parameter tuned
# the parameters that we got above are found using grid search

y_pred = regr1.predict(df_test)
rndf_test_predictions = [round(value) for value in y_pred]
y_pred = regr1.predict(df_train)
rndf_train_predictions = [round(value) for value in y_pred]

In [105]:
test_predict = regr1.predict(input_list[3])
test_predict

array([12.57678755])

In [100]:
input_list[0]['PULocationID']

4번지역    4
Name: PULocationID, dtype: int64

In [124]:
# arr = np.array([])
# arr = np.append(arr, np.array([1, 2, 3]))
# arr = np.append(arr, np.array([4, 5]))
# print(arr)

jinsu_column = ['OBJECTID', 'predict_number']
predict_df = pd.DataFrame(columns = jinsu_column)
jinsu_index = 0
for i in input_list:
    temp_result = np.array([i['PULocationID'].iloc[0]])
    temp_predict_value = regr1.predict(i)
    temp_result = np.append(temp_result, temp_predict_value)
    predict_df.loc[jinsu_index] = temp_result
    jinsu_index += 1

print(predict_df)


    OBJECTID  predict_number
0        4.0        3.072544
1       12.0        4.886163
2       13.0       18.021081
3       24.0       12.576788
4       41.0       23.861578
..       ...             ...
64     246.0       73.614238
65     249.0       75.146123
66     261.0       41.332588
67     262.0       38.001822
68     263.0       56.370002

[69 rows x 2 columns]


In [101]:
df_test

,ft_5,ft_4,ft_3,ft_2,ft_1,PULocationID,weekday,exp_avg
0,1,1,1,2,2,1,5,1
1,1,1,2,2,2,1,5,1
2,1,2,2,2,2,1,5,1
3,2,2,2,2,4,1,5,3
4,2,2,2,4,4,1,5,3
...,...,...,...,...,...,...,...,...
1087918,37,40,32,53,39,69,2,41
1087919,40,32,53,39,28,69,2,31
1087920,32,53,39,28,33,69,2,32
1087921,53,39,28,33,30,69,2,30


In [40]:
y_pred

array([ 8.35347556,  9.57000534, 10.36064495, ...,  3.9087823 ,
        2.48008677,  5.08856498])

In [41]:
#feature importances based on analysis using random forest
print (df_train.columns)
print (regr1.feature_importances_)

Index(['ft_5', 'ft_4', 'ft_3', 'ft_2', 'ft_1', 'PULocationID', 'weekday',
       'exp_avg'],
      dtype='object')
[0.07684949 0.07512184 0.18006786 0.13625224 0.23265497 0.01096237
 0.00264471 0.28544652]


In [42]:
df_test

,ft_5,ft_4,ft_3,ft_2,ft_1,PULocationID,weekday,exp_avg
0,1,1,1,2,2,1,5,1
1,1,1,2,2,2,1,5,1
2,1,2,2,2,2,1,5,1
3,2,2,2,2,4,1,5,3
4,2,2,2,4,4,1,5,3
...,...,...,...,...,...,...,...,...
1087918,37,40,32,53,39,69,2,41
1087919,40,32,53,39,28,69,2,31
1087920,32,53,39,28,33,69,2,32
1087921,53,39,28,33,30,69,2,30


In [43]:
rndf_train_predictions

[8,
 10,
 10,
 12,
 12,
 14,
 14,
 16,
 16,
 18,
 19,
 17,
 14,
 15,
 13,
 11,
 13,
 13,
 13,
 13,
 14,
 15,
 13,
 12,
 8,
 8,
 5,
 6,
 5,
 5,
 4,
 5,
 4,
 3,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 2,
 3,
 3,
 4,
 3,
 3,
 3,
 2,
 2,
 1,
 1,
 1,
 2,
 3,
 4,
 4,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 2,
 3,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 4,
 4,
 5,
 5,
 4,
 4,
 3,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 2,
 3,
 3,
 2,
 3,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 3,
 3,
 2,
 2,
 3,
 3,
 

## Using XgBoost Regressor

In [44]:
# Training a hyper-parameter tuned Xg-Boost regressor on our train data

# find more about XGBRegressor function here http://xgboost.readthedocs.io/en/latest/python/python_api.html?#module-xgboost.sklearn
# -------------------------
# default paramters
# xgboost.XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='reg:linear', 
# booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, 
# colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, 
# missing=None, **kwargs)

# some of methods of RandomForestRegressor()
# fit(X, y, sample_weight=None, eval_set=None, eval_metric=None, early_stopping_rounds=None, verbose=True, xgb_model=None)
# get_params([deep])	Get parameters for this estimator.
# predict(data, output_margin=False, ntree_limit=0) : Predict with data. NOTE: This function is not thread safe.
# get_score(importance_type='weight') -> get the feature importance
# -----------------------
# video link1: https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/regression-using-decision-trees-2/
# video link2: https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/what-are-ensembles/
# -----------------------

x_model = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)
x_model.fit(df_train, tsne_train_output)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=3,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=4, nthread=4, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=200, ...)

In [45]:
#predicting with our trained Xg-Boost regressor
# the models x_model is already hyper parameter tuned
# the parameters that we got above are found using grid search

y_pred = x_model.predict(df_test)
xgb_test_predictions = [round(value) for value in y_pred]
y_pred = x_model.predict(df_train)
xgb_train_predictions = [round(value) for value in y_pred]

In [46]:
#feature importances
x_model.get_booster().get_score(importance_type="weight")

{'ft_5': 945.0,
 'ft_4': 745.0,
 'ft_3': 813.0,
 'ft_2': 970.0,
 'ft_1': 1122.0,
 'PULocationID': 1153.0,
 'weekday': 189.0,
 'exp_avg': 994.0}

## Calculating the error metric values for various models

In [47]:
train_R2=[]
test_R2=[]

train_R2.append(r2_score(tsne_train_output,df_train['ft_1'].values))
train_R2.append(r2_score(tsne_train_output,df_train['exp_avg'].values))
train_R2.append(r2_score(tsne_train_output,rndf_train_predictions))
train_R2.append(r2_score(tsne_train_output,xgb_train_predictions))
train_R2.append(r2_score(tsne_train_output,lr_train_predictions))

test_R2.append(r2_score(tsne_test_output,df_test['ft_1'].values))
test_R2.append(r2_score(tsne_test_output,df_test['exp_avg'].values))
test_R2.append(r2_score(tsne_test_output,rndf_test_predictions))
test_R2.append(r2_score(tsne_test_output,xgb_test_predictions))
test_R2.append(r2_score(tsne_test_output,lr_test_predictions))

In [48]:
print(f"{base_year+1} 1 ~ {base_month_count} {region} R2")
print ("--------------------------------------------------------------------------------------------------------")
print ("Baseline Model\t\t\t","Train: ",train_R2[0],"Test: ",test_R2[0],sep='\t')
print ("Exponential Averages Forecasting","Train: ",train_R2[1],"Test: ",test_R2[1],sep='\t')
print ("Random Forest Regression\t","Train: ",train_R2[2],"Test: ",test_R2[2],sep='\t')
print ("XgBoost Regression\t\t","Train: ",train_R2[3],"Test: ",test_R2[3],sep='\t')
print ("Linear Regression\t\t","Train: ",train_R2[4],"Test: ",test_R2[4],sep='\t')

2019 1 ~ 12 Manhattan R2
--------------------------------------------------------------------------------------------------------
Baseline Model				Train: 	0.9261341021248126	Test: 	0.9279697110296186
Exponential Averages Forecasting	Train: 	0.9353202829543953	Test: 	0.93690114529243
Random Forest Regression		Train: 	0.9642250369647256	Test: 	0.9404143153398121
XgBoost Regression			Train: 	0.9399804186815103	Test: 	0.9409247248608636
Linear Regression			Train: 	0.937148019499591	Test: 	0.9386946289815727


In [49]:
train_MAE=[]
test_MAE=[]
train_MAE.append(mean_absolute_error(tsne_train_output,df_train['ft_1'].values))
train_MAE.append(mean_absolute_error(tsne_train_output,df_train['exp_avg'].values))
train_MAE.append(mean_absolute_error(tsne_train_output,rndf_train_predictions))
train_MAE.append(mean_absolute_error(tsne_train_output,xgb_train_predictions))
train_MAE.append(mean_absolute_error(tsne_train_output,lr_train_predictions))

test_MAE.append(mean_absolute_error(tsne_test_output,df_test['ft_1'].values))
test_MAE.append(mean_absolute_error(tsne_test_output,df_test['exp_avg'].values))
test_MAE.append(mean_absolute_error(tsne_test_output,rndf_test_predictions))
test_MAE.append(mean_absolute_error(tsne_test_output,xgb_test_predictions))
test_MAE.append(mean_absolute_error(tsne_test_output,lr_test_predictions))

In [50]:
print(f"{base_year+1} 1 ~ {base_month_count} {region} MAE")
print ("--------------------------------------------------------------------------------------------------------")
print ("Baseline Model\t\t\t","Train: ",train_MAE[0],"Test: ",test_MAE[0],sep='\t')
print ("Exponential Averages Forecasting","Train: ",train_MAE[1],"Test: ",test_MAE[1],sep='\t')
print ("Random Forest Regression\t","Train: ",train_MAE[2],"Test: ",test_MAE[2],sep='\t')
print ("XgBoost Regression\t\t","Train: ",train_MAE[3],"Test: ",test_MAE[3],sep='\t')
print ("Linear Regression\t\t","Train: ",train_MAE[4],"Test: ",test_MAE[4],sep='\t')

2019 1 ~ 12 Manhattan MAE
--------------------------------------------------------------------------------------------------------
Baseline Model				Train: 	4.155000921850698	Test: 	4.0590041758470035
Exponential Averages Forecasting	Train: 	3.8772323363163474	Test: 	3.787620998912607
Random Forest Regression		Train: 	2.9204738312587755	Test: 	3.7134668538122644
XgBoost Regression			Train: 	3.7752823463227614	Test: 	3.6963663788705636
Linear Regression			Train: 	3.8324524537774605	Test: 	3.743439563277916
